<a href="https://colab.research.google.com/github/aryan-cs/poker-like-games/blob/discrete-poker/discrete_poker_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup
Run this cell to set up some helpful tools.

In [15]:
import time
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from rich.console import Console
from rich.table import Table

log_data = []
P1 = True
P2 = True

# First player has n options, second player has n - 1 options (picking without replacement)
def nCr(n): return n * (n - 1) # / 2

# Saving -- irrelevant
def log(p1_cutoff, p2_cutoff, p1_expected_payout, p2_expected_payout, time_taken, verbose=True):
    log_data.append({
        'p1_cutoff': p1_cutoff,
        'p2_cutoff': p2_cutoff,
        'p1_expected_payout': p1_expected_payout,
        'p2_expected_payout': p2_expected_payout,
        'time_taken': time_taken
    })

    message = '''
P1 Cutoff: {0}
P2 Cutoff: {1}
P1 Payout: {2:.5f}
P2 Payout: {3:.5f}
Executed in {4:.5f} seconds
'''.format(p1_cutoff,
            p2_cutoff,
            p1_expected_payout,
            p2_expected_payout,
            time_taken)

    if verbose: boxed(message)

# Visual appeal -- irrelevant
def boxed(text):
    lines = text.splitlines()
    if lines and lines[0] == "": lines = lines[1:]
    if not lines: return

    max_width = max(map(len, lines))

    print("┌" + "─" * (max_width + 2) + "┐")
    for line in lines:
      print(f"│ {line.ljust(max_width)} │")
    print("└" + "─" * (max_width + 2) + "┘")

# Graphing -- irrelevant
def graph_3D(p1=False, p2=False):
    df = pd.DataFrame(log_data)
    fig = go.Figure()

    if p1:
        fig.add_trace(go.Scatter3d(
            x=df['p1_cutoff'],
            y=df['p2_cutoff'],
            z=df['p1_expected_payout'],
            mode='markers',
            marker=dict(color='blue', size=4),
            name='Player 1'
        ))

    if p2:
        fig.add_trace(go.Scatter3d(
            x=df['p1_cutoff'],
            y=df['p2_cutoff'],
            z=df['p2_expected_payout'],
            mode='markers',
            marker=dict(color='red', size=4),
            name='Player 2'
        ))

    fig.update_layout(
        title='Payouts as a function of Cutoffs | Scatter',
        scene=dict(
            xaxis_title='P1 Cutoff',
            yaxis_title='P2 Cutoff',
            zaxis_title='Payout'
        ),
        showlegend=True
    )

    fig.show()

# Graphing -- irrelevant
def graph_contour(p1=False, p2=False):
    df = pd.DataFrame(log_data).groupby(['p1_cutoff', 'p2_cutoff'], as_index=False).mean()
    fig = go.Figure()
    x = np.unique(df['p1_cutoff'])
    y = np.unique(df['p2_cutoff'])
    X, Y = np.meshgrid(x, y)

    if p1:
        Z1 = df.pivot_table(
            index='p2_cutoff',
            columns='p1_cutoff',
            values='p1_expected_payout').values
        fig.add_trace(go.Surface(x=X, y=Y, z=Z1,
                                 colorscale='Blues',
                                 opacity=0.9,
                                 name="Player 1 Payout"))

    if p2:
        Z2 = df.pivot_table(
            index='p2_cutoff',
            columns='p1_cutoff',
            values='p2_expected_payout').values
        fig.add_trace(go.Surface(x=X, y=Y, z=Z2,
                                 colorscale='Reds',
                                 opacity=0.9,
                                 name="Player 2 Payout"))

    fig.update_layout(
        title='Payouts as a function of Cutoffs | Contour',
        scene=dict(
            xaxis_title='P1 Cutoff',
            yaxis_title='P2 Cutoff',
            zaxis_title='Payout'
        ),
        showlegend=True
    )
    fig.show()

def printTable(p1=False, p2=False):
    console = Console()
    df = pd.DataFrame(log_data).groupby(['p1_cutoff', 'p2_cutoff'], as_index=False).mean()

    if p1:
        pivot = df.pivot(index='p1_cutoff', columns='p2_cutoff', values='p1_expected_payout')
        title = "P1 Expected Payout"
    elif p2:
        pivot = df.pivot(index='p1_cutoff', columns='p2_cutoff', values='p2_expected_payout')
        title = "P2 Expected Payout"
    else:
        print("Please specify either p1=True or p2=True")
        return

    table = Table(title=title, show_lines=True, padding=(0,1), collapse_padding=True)

    # Add column for P1 Cutoff
    table.add_column("P1 Cutoff/P2 Cutoff", style="cyan", justify="right")

    # Add columns for each P2 Cutoff
    for col in pivot.columns:
        table.add_column(f"{col}", style="green", justify="right")

    # Add rows
    for idx, row in pivot.iterrows():
        table_row = [f"{idx:.2f}"] + [f"{val:.5f}" if pd.notnull(val) else "-" for val in row]
        table.add_row(*table_row, end_section=True)
    console.print(table)
def printFormulaTable(data,p1=False,p2=False):
    console = Console()
    df = pd.DataFrame(data)
    if p1:
        pivot = df.pivot(index='p1_cutoff', columns='p2_cutoff', values='p1_expected_payout')
        title = "P1 Payout from Formula"
    elif p2:
        pivot = df.pivot(index='p1_cutoff', columns='p2_cutoff', values='p2_expected_payout')
        title = "P2 Payout from Formula"
    else:
        print("Please specify either p1=True or p2=True")
        return
    table = Table(title=title, show_lines=True, padding=(0,1), collapse_padding=True)
    # Add column for P1 Cutoff
    table.add_column("P1 Cutoff/P2 Cutoff", style="cyan", justify="right")

    # Add columns for each P2 Cutoff
    for col in pivot.columns:
        table.add_column(f"{col:.2f}", style="green", justify="right")

    # Add rows
    for idx, row in pivot.iterrows():
        table_row = [f"{idx:.2f}"] + [f"{val:.5f}" if pd.notnull(val) else "-" for val in row]
        table.add_row(*table_row, end_section=True)

    console.print(table)

# Exporting -- irrelevant
def save():
    df = pd.DataFrame(log_data)
    df.to_csv('data.csv', index=False)
    print("\nLog data saved.")


# 1. Simulation
Simulating payoffs for various cutoff combinations.

In [20]:
#@title { vertical-output: true }

"""

  Example n values:
    n = 3: Ace/King/Queen
    n = 6: Dice
    n = 13: Full Suit Poker
    n = 1000: Roughly continuous

  Iterating for cutoff ranges of [1, test_range]

"""
n = 10
test_range = n
p1_expected_payout = 0
p2_expected_payout = 0
trials = nCr(n)
def test_cutoffs(p1_cutoff, p2_cutoff):

  start_time = time.time()

  global p1_expected_payout
  global p2_expected_payout

  p1_expected_payout = 0
  p2_expected_payout = 0

  # Testing every possible 'card' for P1 and P2
  for p1_card in range(1, n + 1):
    for p2_card in range(1, n + 1):

      if p1_card == p2_card: continue

      # P1 in betting region, P2 in betting region
      if p1_card >= p1_cutoff and p2_card >= p2_cutoff:

        if p1_card > p2_card:       # P1 wins showdown
          p1_expected_payout += 2
          p2_expected_payout -= 2
        else:                       # P2 wins showdown
          p1_expected_payout -= 2
          p2_expected_payout += 2

      # P1 in folding region, P2 in folding region
      elif p1_card < p1_cutoff and p2_card < p2_cutoff:

        if p1_card > p2_card:       # P1 wins showdown
          p1_expected_payout += 1
          p2_expected_payout -= 1
        else:                       # P2 wins showdown
          p1_expected_payout -= 1
          p2_expected_payout += 1

      # P1 in betting region, P2 in folding region | P2 forfeits, P1 wins be default
      elif p1_card >= p1_cutoff:
        p1_expected_payout += 1
        p2_expected_payout -= 1

      # P1 in folding region, P2 in betting region | P1 forfeits, P2 wins be default
      elif p2_card >= p2_cutoff:
        p1_expected_payout -= 1
        p2_expected_payout += 1

  return time.time() - start_time

log_data = []
start_time = time.time()
formula_data = pd.DataFrame()
data = []
# Testing all possible cutoffs
for p1_test_cutoff in range(1, test_range + 2):
  for p2_test_cutoff in range(1, test_range + 2):
    timeTaken = test_cutoffs(p1_test_cutoff, p2_test_cutoff)
    log(p1_test_cutoff,
        p2_test_cutoff,
        p1_expected_payout / trials,
        p2_expected_payout / trials,
        timeTaken,
        verbose=False)
    formula = 0
    if p2_test_cutoff > p1_test_cutoff: p1_payout = 2*p2_test_cutoff**2-(n+2+3*p1_test_cutoff)*p2_test_cutoff+(p1_test_cutoff**2+(n+2)*p1_test_cutoff)
    else: p1_payout = -2*p1_test_cutoff**2+(n+2+3*p2_test_cutoff)*p1_test_cutoff-(p2_test_cutoff**2+(n+2)*p2_test_cutoff)
    p1_payout /= trials
    data.append({
                'p1_cutoff': p1_test_cutoff,
                'p2_cutoff': p2_test_cutoff,
                'p1_expected_payout': p1_payout,
                'p2_expected_payout': -p1_payout
            })

print(f"Executed all iterations in {round(time.time() - start_time, 3)} seconds.\n")
# Add flags as necessary (P1, P2)
#graph_3D(P1)
#graph_contour(P1)
printTable(P1) # Warning: Do not use for high values of n (n>=20)
printFormulaTable(data,P1)
save()

Executed all iterations in 0.004 seconds.



                                                P1 Expected Payout                                                 
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃      P1 ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃        ┃
┃ Cutoff… ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃        ┃
┃  Cutoff ┃       1 ┃      2 ┃       3 ┃      4 ┃       5 ┃      6 ┃       7 ┃      8 ┃       9 ┃     10 ┃     11 ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│    1.00 │ 0.00000 │-0.100… │-0.15556 │-0.166… │-0.13333 │-0.055… │ 0.06667 │0.23333 │ 0.44444 │0.70000 │1.00000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    2.00 │ 0.10000 │0.00000 │-0.08889 │-0.133… │-0.13333 │-0.088… │ 0.00000 │0.13333 │ 0.31111 │0.53333 │0.80000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    3.00 │ 0.15556 │0.08889 │ 0.00000 │-0.077… │-0.11111 │-0.100… │-0.04444 │0.05556 │ 0.20000 │0.38889 │0.62222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    4.00 │ 0.16667 │0.13333 │ 0.07778 │0.00000 │-0.06667 │-0.088… │-0.06667 │0.00000 │ 0.11111 │0.26667 │0.46667 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    5.00 │ 0.13333 │0.13333 │ 0.11111 │0.06667 │ 0.00000 │-0.055… │-0.06667 │-0.033… │ 0.04444 │0.16667 │0.33333 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    6.00 │ 0.05556 │0.08889 │ 0.10000 │0.08889 │ 0.05556 │0.00000 │-0.04444 │-0.044… │ 0.00000 │0.08889 │0.22222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    7.00 │-0.06667 │0.00000 │ 0.04444 │0.06667 │ 0.06667 │0.04444 │ 0.00000 │-0.033… │-0.02222 │0.03333 │0.13333 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    8.00 │-0.23333 │-0.133… │-0.05556 │0.00000 │ 0.03333 │0.04444 │ 0.03333 │0.00000 │-0.02222 │0.00000 │0.06667 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    9.00 │-0.44444 │-0.311… │-0.20000 │-0.111… │-0.04444 │0.00000 │ 0.02222 │0.02222 │ 0.00000 │-0.011… │0.02222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│   10.00 │-0.70000 │-0.533… │-0.38889 │-0.266… │-0.16667 │-0.088… │-0.03333 │0.00000 │ 0.01111 │0.00000 │0.00000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│   11.00 │-1.00000 │-0.800… │-0.62222 │-0.466… │-0.33333 │-0.222… │-0.13333 │-0.066… │-0.02222 │0.00000 │0.00000 │
└─────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴────────┘

                                              P1 Payout from Formula                                               
┏━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃      P1 ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃        ┃
┃ Cutoff… ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃         ┃        ┃        ┃
┃  Cutoff ┃    1.00 ┃   2.00 ┃    3.00 ┃   4.00 ┃    5.00 ┃   6.00 ┃    7.00 ┃   8.00 ┃    9.00 ┃  10.00 ┃  11.00 ┃
┡━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│    1.00 │ 0.00000 │-0.100… │-0.15556 │-0.166… │-0.13333 │-0.055… │ 0.06667 │0.23333 │ 0.44444 │0.70000 │1.00000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    2.00 │ 0.10000 │0.00000 │-0.08889 │-0.133… │-0.13333 │-0.088… │ 0.00000 │0.13333 │ 0.31111 │0.53333 │0.80000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    3.00 │ 0.15556 │0.08889 │ 0.00000 │-0.077… │-0.11111 │-0.100… │-0.04444 │0.05556 │ 0.20000 │0.38889 │0.62222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    4.00 │ 0.16667 │0.13333 │ 0.07778 │0.00000 │-0.06667 │-0.088… │-0.06667 │0.00000 │ 0.11111 │0.26667 │0.46667 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    5.00 │ 0.13333 │0.13333 │ 0.11111 │0.06667 │ 0.00000 │-0.055… │-0.06667 │-0.033… │ 0.04444 │0.16667 │0.33333 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    6.00 │ 0.05556 │0.08889 │ 0.10000 │0.08889 │ 0.05556 │0.00000 │-0.04444 │-0.044… │ 0.00000 │0.08889 │0.22222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    7.00 │-0.06667 │0.00000 │ 0.04444 │0.06667 │ 0.06667 │0.04444 │ 0.00000 │-0.033… │-0.02222 │0.03333 │0.13333 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    8.00 │-0.23333 │-0.133… │-0.05556 │0.00000 │ 0.03333 │0.04444 │ 0.03333 │0.00000 │-0.02222 │0.00000 │0.06667 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│    9.00 │-0.44444 │-0.311… │-0.20000 │-0.111… │-0.04444 │0.00000 │ 0.02222 │0.02222 │ 0.00000 │-0.011… │0.02222 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│   10.00 │-0.70000 │-0.533… │-0.38889 │-0.266… │-0.16667 │-0.088… │-0.03333 │0.00000 │ 0.01111 │0.00000 │0.00000 │
├─────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼─────────┼────────┼────────┤
│   11.00 │-1.00000 │-0.800… │-0.62222 │-0.466… │-0.33333 │-0.222… │-0.13333 │-0.066… │-0.02222 │0.00000 │0.00000 │
└─────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴─────────┴────────┴────────┘


Log data saved.
